In [1]:
import numpy as np
import pandas as pd

In [58]:
i5 = pd.read_excel('./test_data/iTru/new_iTru_index_list.xlsx',
               sheetname='iTru5')

i7 = pd.read_excel('./test_data/iTru/new_iTru_index_list.xlsx',
               sheetname='iTru7')

In [47]:
i5.head()

,count,Primer Name,Primer Index Sequence,MiSeq & HiSeq Index Sequence,MiniSeq & NextSeq Index Sequence,Row,Col
0,1,iTru5_01_A,ACCGACAA,ACCGACAA,TTGTCGGT,A,1
1,2,iTru5_01_B,AGTGGCAA,AGTGGCAA,TTGCCACT,B,1
2,3,iTru5_01_C,CACAGACT,CACAGACT,AGTCTGTG,C,1
3,4,iTru5_01_D,CGACACTT,CGACACTT,AAGTGTCG,D,1
4,5,iTru5_01_E,GACTTGTG,GACTTGTG,CACAAGTC,E,1


In [48]:
i7.head()

,count,Primer Name,Primer Index Sequence,Row,Col
0,1,iTru7_101_01,ACGTTACC,A,1
1,2,iTru7_101_02,CTGTGTTG,A,2
2,3,iTru7_101_03,TGAGGTGT,A,3
3,4,iTru7_101_04,GATCCATG,A,4
4,5,iTru7_101_05,GCCTATCA,A,5


In [54]:
combos = 384 * 384
i5_name = [''] * combos
i7_name = [''] * combos
i5_seq = [''] * combos
i7_seq = [''] * combos
i57_seq = [''] * combos
i5_well = [''] * combos
i7_well = [''] * combos
i5_plate = ['iTru5_plate'] * combos
i7_plate = ['iTru7_plate'] * combos
idx = [np.nan] * combos

i = 0
for i7_offset in range(384):
    for x in range(384):
        i5_idx = x
        i7_idx = (x + i7_offset) % 384
        idx[i] = i
        i5_name[i] = i5.iloc[i5_idx]['Primer Name']
        i7_name[i] = i7.iloc[i7_idx]['Primer Name']
        i5_seq[i] =  i5.iloc[i5_idx]['Primer Index Sequence']
        i7_seq[i] = i7.iloc[i7_idx]['Primer Index Sequence']
        i57_seq[i] = i5_seq[i] + i7_seq[i]
        i5_well[i] =  '{0}{1}'.format(i5.iloc[i5_idx]['Row'],i5.iloc[i5_idx]['Col'])
        i7_well[i] = '{0}{1}'.format(i7.iloc[i7_idx]['Row'],i7.iloc[i7_idx]['Col'])
        i += 1

index_combo_df = pd.DataFrame(np.array([idx, i57_seq, i5_name, i5_seq, i5_well, i5_plate, i7_name, i7_seq, i7_well, i7_plate]).T,
                              columns = ['index combo','index combo seq','i5 name','i5 sequence','i5 well','i5 plate',
                                         'i7 name','i7 sequence','i7 well','i7 plate'])

In [55]:
index_combo_df

,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,0,ACCGACAAACGTTACC,iTru5_01_A,ACCGACAA,A1,iTru5_plate,iTru7_101_01,ACGTTACC,A1,iTru7_plate
1,1,AGTGGCAACTGTGTTG,iTru5_01_B,AGTGGCAA,B1,iTru5_plate,iTru7_101_02,CTGTGTTG,A2,iTru7_plate
2,2,CACAGACTTGAGGTGT,iTru5_01_C,CACAGACT,C1,iTru5_plate,iTru7_101_03,TGAGGTGT,A3,iTru7_plate
3,3,CGACACTTGATCCATG,iTru5_01_D,CGACACTT,D1,iTru5_plate,iTru7_101_04,GATCCATG,A4,iTru7_plate
4,4,GACTTGTGGCCTATCA,iTru5_01_E,GACTTGTG,E1,iTru5_plate,iTru7_101_05,GCCTATCA,A5,iTru7_plate
5,5,GTGAGACTAACAACCG,iTru5_01_F,GTGAGACT,F1,iTru5_plate,iTru7_101_06,AACAACCG,A6,iTru7_plate
6,6,GTTCCATGACTCGTTG,iTru5_01_G,GTTCCATG,G1,iTru5_plate,iTru7_101_07,ACTCGTTG,A7,iTru7_plate
7,7,TAGCTGAGCCTATGGT,iTru5_01_H,TAGCTGAG,H1,iTru5_plate,iTru7_101_08,CCTATGGT,A8,iTru7_plate
8,8,CTTCGCAATGTACACC,iTru5_02_A,CTTCGCAA,I1,iTru5_plate,iTru7_101_09,TGTACACC,A9,iTru7_plate
9,9,GTGGTATGGTATGCTG,iTru5_02_B,GTGGTATG,J1,iTru5_plate,iTru7_101_10,GTATGCTG,A10,iTru7_plate


In [56]:
len(set(index_combo_df['index combo seq']))

147456

In [57]:
index_combo_df.to_csv('./test_output/iTru/new_iTru_combos.csv', index=False)